In [103]:
# Video: https://www.youtube.com/watch?v=2JDR6jv0fGA
import pandas as pd
import numpy as np

seasons = [str(season) for season in range(2013, 2023)]

full_team_names = {
    'crd': 'Arizona Cardinals', 'atl': 'Atlanta Falcons', 'rav': 'Baltimore Ravens', 'buf': 'Buffalo Bills', 'car': 'Carolina Panthers', 
    'chi': 'Chicago Bears', 'cin': 'Cincinnati Bengals', 'cle': 'Cleveland Browns', 'dal': 'Dallas Cowboys', 'den': 'Denver Broncos', 
    'det': 'Detroit Lions', 'gnb': 'Green Bay Packers', 'htx': 'Houston Texans', 'clt': 'Indianapolis Colts', 'jax': 'Jacksonville Jaguars', 
    'kan': 'Kansas City Chiefs', 'sdg': 'Los Angeles Chargers', 'ram': 'Los Angeles Rams', 'rai': 'Las Vegas Raiders', 'mia': 'Miami Dolphins', 
    'min': 'Minnesota Vikings', 'nwe': 'New England Patriots', 'nor': 'New Orleans Saints', 'nyg': 'New York Giants', 'nyj': 'New York Jets', 
    'phi': 'Philadelphia Eagles', 'pit': 'Pittsburgh Steelers', 'sea': 'Seattle Seahawks', 'sfo': 'San Francisco 49ers', 'tam': 'Tampa Bay Buccaneers', 'oti': 'Tennessee Titans', 'was': 'Washington Commanders'
}
teams = list(full_team_names.keys())
print(len(seasons))

10


In [104]:
import random
import time
import os

# code block to get the initial game boxscores from PFR and convert it to a csv file
boxscore_df = pd.DataFrame()
folder_path = 'csv_files'
file_name = 'gamelogs_2013-2023.csv'

if not os.path.exists(folder_path):
    os.makedirs(folder_path)

file_path = os.path.join(folder_path, file_name)

if not os.path.exists(file_path):
    for season in seasons:
        for team in teams:
            # getting the gamelog for every team in every year
            url = "https://www.pro-football-reference.com/teams/" + team + "/" + season + "/gamelog/"
            print(url)
            team_df = pd.read_html(url, header=1, attrs={'id':'gamelog' + season})[0]
            
            team_df.insert(loc=0, column='Season', value=season)
            team_df.insert(loc=2, column='Team', value=full_team_names[team])
            
            boxscore_df = pd.concat([boxscore_df, team_df], ignore_index=True)
            
            time.sleep(random.randint(4, 5))
        
        print(boxscore_df)
    boxscore_df.to_csv(file_path, index=False)
else:
    boxscore_df = pd.read_csv(file_path)

In [105]:
# Cleans up the dataframe, makes sure to name some columns
new_columns = {'Unnamed: 4': 'Win', 'Unnamed: 6':'Home', 'Tm':'Off_Pts', 'Opp.1':'Def_Pts'}
rename_dict = {'St. Louis Rams':'Los Angeles Rams',
               'Oakland Raiders': 'Las Vegas Raiders',
               'San Diego Chargers': 'Los Angeles Chargers',
               'Washington Football Team': 'Washington Commanders',
               'Washington Redskins': 'Washington Commanders'}
boxscore_df = boxscore_df.rename(columns=new_columns)
boxscore_df = boxscore_df.replace(to_replace={'Opp': rename_dict})

In [106]:
# Replacing values in columns for Win and Home
boxscore_df['Win'] = boxscore_df['Win'].map({'W': 1, 'L': 0, 'T': 0.5})
boxscore_df['Home'] = boxscore_df['Home'].map({'': 1, '@': 0})

# Replacing values in the OT column of the dataframe
boxscore_df['OT'] = boxscore_df['OT'].map({'OT': 1, '': 0})

In [107]:
matchup_df = boxscore_df[:]
print(matchup_df.columns)
# Creating a new dataframe to use

Index(['Season', 'Week', 'Team', 'Day', 'Date', 'Unnamed: 3', 'Win', 'OT',
       'Home', 'Opp', 'Off_Pts', 'Def_Pts', 'Cmp', 'Att', 'Yds', 'TD', 'Int',
       'Sk', 'Yds.1', 'Y/A', 'NY/A', 'Cmp%', 'Rate', 'Att.1', 'Yds.2', 'Y/A.1',
       'TD.1', 'FGM', 'FGA', 'XPM', 'XPA', 'Pnt', 'Yds.3', '3DConv', '3DAtt',
       '4DConv', '4DAtt', 'ToP'],
      dtype='object')


In [108]:
# Dropping columns which would be available after the game in this cell
matchup_df = matchup_df.drop(matchup_df.columns[10:], axis=1)
matchup_df = matchup_df.drop(matchup_df.columns[[5, 7]], axis=1)

In [109]:
print(matchup_df.columns)

Index(['Season', 'Week', 'Team', 'Day', 'Date', 'Win', 'Home', 'Opp'], dtype='object')


In [110]:
print(matchup_df)

      Season  Week                   Team  Day          Date  Win  Home  \
0       2013     1      Arizona Cardinals  Sun   September 8  0.0   0.0   
1       2013     2      Arizona Cardinals  Sun  September 15  1.0   NaN   
2       2013     3      Arizona Cardinals  Sun  September 22  0.0   0.0   
3       2013     4      Arizona Cardinals  Sun  September 29  1.0   0.0   
4       2013     5      Arizona Cardinals  Sun     October 6  1.0   NaN   
...      ...   ...                    ...  ...           ...  ...   ...   
5177    2022    13  Washington Commanders  Sun    December 4  0.5   0.0   
5178    2022    15  Washington Commanders  Sun   December 18  0.0   NaN   
5179    2022    16  Washington Commanders  Sat   December 24  0.0   0.0   
5180    2022    17  Washington Commanders  Sun     January 1  0.0   NaN   
5181    2022    18  Washington Commanders  Sun     January 8  1.0   NaN   

                       Opp  
0         Los Angeles Rams  
1            Detroit Lions  
2       New 

In [111]:
# Will need to implement so it updates weekly

# https://www.pro-football-reference.com/teams/mia/2023.htm#team_stats
# Example link for team rankings

import random
import time
import os

# code block to get the team rankings(Offense/Defense) from PFR and convert it to a csv file
all_tm_rkgs_df = pd.DataFrame()
folder_path = 'csv_files'
file_name = 'rankings_2013-2023.csv'

if not os.path.exists(folder_path):
    os.makedirs(folder_path)

file_path = os.path.join(folder_path, file_name)

if not os.path.exists(file_path):
    for season in seasons:
        for team in teams:
            rks_url = "https://www.pro-football-reference.com/teams/" + team + "/" + season + ".htm#team_stats"
            print(rks_url)
            tm_rk_df = pd.read_html(rks_url, header=1, attrs={'id':'team_stats'})[0]
            cvn_url = "https://www.pro-football-reference.com/teams/nwe/2023.htm#team_conversions"
            cvn_df = pd.read_html(rks_url, header=1, attrs={'id':'team_conversions'})[0]
            cvn_df = cvn_df.drop([0, 1])
            cvn_df = cvn_df.drop(['Player'], axis=1)
            rkgs_df = pd.concat([tm_rk_df, cvn_df], axis=1)
            new_columns = {'Yds': 'Tot_Yds', 'Att': 'Pass_Att', 'Yds.1': 'Pass_Yds', 'TD': 'Pass_TD', 'Att.1': 'Rush_Att', 'Yds.2': 'Rush_Yds', 'TD.1': 'Rush_TD', 'Yds.4': 'Avg_Drive_Yds'}
            rkgs_df = rkgs_df.rename(columns=new_columns)
            rkgs_df = rkgs_df.drop([0, 1])
            rkgs_df = rkgs_df.dropna(axis=1, how='all')

            of_columns = [val + '_OF' for val in rkgs_df.columns[1:]]
            df_columns = [val + '_DF' for val in rkgs_df.columns[1:]]
            tot_columns = of_columns + df_columns
            rnk_row = []
            for i, j in rkgs_df.iterrows():
                for k in j:
                    if not isinstance(k, str):
                        rnk_row.append(k)
                    elif k.isnumeric():
                        rnk_row.append(float(k))
            rnk_rows = np.array(rnk_row)
            com_rkgs_df = pd.DataFrame(rnk_rows, tot_columns)
            com_rkgs_df = com_rkgs_df.T
            com_rkgs_df.insert(loc=0, column='Season', value=season) # replace with 2023 for testing
            com_rkgs_df.insert(loc=1, column='Team', value=full_team_names[team]) # Replace with 'New England Patriots' for testing
            
            all_tm_rkgs_df = pd.concat([all_tm_rkgs_df, com_rkgs_df], ignore_index=True)
            
            time.sleep(random.randint(4, 5))
        
        print(com_rkgs_df)
    all_tm_rkgs_df.to_csv(file_path, index=False)
else:
    all_tm_rkgs_df = pd.read_csv(file_path)

In [118]:
merged_df_1 = pd.merge(matchup_df, all_tm_rkgs_df, on=['Season', 'Team'], how='inner')
merged_df_2 = pd.merge(merged_df_1, all_tm_rkgs_df, left_on=['Season', 'Opp'], right_on=['Season', 'Team'], suffixes=('', '_Opp'))
print(merged_df_2)

      Season  Week                   Team  Day          Date  Win  Home  \
0       2013     1      Arizona Cardinals  Sun   September 8  0.0   0.0   
1       2013    14      Arizona Cardinals  Sun    December 8  1.0   NaN   
2       2013     2        Atlanta Falcons  Sun  September 15  1.0   NaN   
3       2013     7      Carolina Panthers  Sun    October 20  1.0   NaN   
4       2013    12          Chicago Bears  Sun   November 24  0.0   0.0   
...      ...   ...                    ...  ...           ...  ...   ...   
5177    2022    11    Philadelphia Eagles  Sun   November 20  1.0   0.0   
5178    2022    12    Pittsburgh Steelers  Mon   November 28  1.0   0.0   
5179    2022     4       Tennessee Titans  Sun     October 2  1.0   0.0   
5180    2022     7       Tennessee Titans  Sun    October 23  1.0   NaN   
5181    2022     8  Washington Commanders  Sun    October 30  1.0   0.0   

                     Opp  PF_OF  Tot_Yds_OF  ...  Sc%_DF_Opp  TO%_DF_Opp  \
0       Los Angeles Ram